In [1]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

import utils.dataset_processors as dataset_processors
import numpy as np
import pandas as pd
import re

c:\Users\User\anaconda3\envs\ml_projects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_model_names(model_name):
    return {
        'distillbert': 'distilbert-base-uncased', # 66M
        'xlnet': 'xlnet-base-cased', # 110M
        'bert': 'bert-base-uncased', # 110M
        'roberta': 'roberta-base', # 125M
        'albert': 'albert-base-v2', # 11M
        'electra': 'google/electra-small-discriminator', # 14M
        'big-bird': 'google/bigbird-roberta-base', # 125M
        'longformer': 'allenai/longformer-base-4096' # 149M
    }[model_name]

In [2]:
# Initialize the model name for tokenizer and also the saved model name
model_name = 'distilbert-base-uncased'
saved_model = 'distillbert-finetuned-segmented'

# Add boolean for sentence segmentation:
segment_sentences = False

# Add filename that will be saved later:

In [3]:
# Load the dataset
datafile = "data/essays/essays.csv"
dataset = dataset_processors.load_essays_df(datafile)

# Split the dataset (6:2:2)
train_data, temp_data = train_test_split(dataset, train_size=0.6, random_state=42)
validation_data, test_data = train_test_split(temp_data, train_size=0.5, random_state=42)

In [ ]:
# Get the Big 5 labels
column_names = list(train_data.columns)
labels = [label for label in column_names if label not in ['user','text','token_len']]

# Forward and backward mapping
id2label = {idx:label for idx,label in enumerate(labels)}
label2id = {label:idx for idx,label in enumerate(labels)}

labels

['EXT', 'NEU', 'AGR', 'CON', 'OPN']

In [5]:
# Import the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    problem_type = "multi_label_classification",
    num_labels = len(labels),
    id2label = id2label,
    label2id = label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Convert from essay to sentences
def split_text_with_labels(row):
    
    # Split sentences
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", row['text'])

    return [{
        'text': sentence,
        'EXT': row['EXT'],
        'NEU': row['NEU'],
        'AGR': row['AGR'],
        'CON': row['CON'],
        'OPN': row['OPN']
    }
        for sentence in sentences       
    ]

def transform_dataframe(old_dataframe):

    # Begin the split
    split_data = []
    
    for _, row in old_dataframe.iterrows():
        split_data.extend(split_text_with_labels(row))
        
    return pd.DataFrame(split_data)

if segment_sentences:
    train_data = transform_dataframe(train_data)
    test_data = transform_dataframe(test_data)
    validation_data = transform_dataframe(validation_data)

In [7]:
print(train_data.head(20))

                 user                                               text  \
1244  2000_691500.txt  I am tired now. I don't know what I should tal...   
579   1998_894233.txt  I never thought that college would be this ove...   
1660  2002_651875.txt       Well, my first psychology writing assignm...   
1456  2000_940609.txt  I swear that office space is the funniest movi...   
892   1999_821141.txt  Why is'nt my roomate quieter doesn't he unders...   
2124     2003_349.txt  As I look at my clock in the lower right hand ...   
780   1999_633874.txt  We went to Barton Springs today as a sorority,...   
2052     2003_223.txt  I can overhear the sound of my roommate watchi...   
159   1997_936626.txt  It is now 12:32 and so I cannot wait until 12:...   
1047  2000_075257.txt  I'm so excited that my computer is doing what ...   
2127     2003_357.txt  Wow, that clock starts right off the bat. I al...   
1444  2000_933743.txt  Last night around 11pm or so a friend of mine ...   
2017     200

In [8]:
# Convert to DatasetDict
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
valid_dataset = Dataset.from_dict(validation_data)

full_dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "valid": valid_dataset
})

print(full_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['user', 'text', 'token_len', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 1480
    })
    test: Dataset({
        features: ['user', 'text', 'token_len', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 494
    })
    valid: Dataset({
        features: ['user', 'text', 'token_len', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 493
    })
})


In [9]:
def preprocess_text(row):

    # Extract the text
    essays = row['text']

    # Clean up
    essays = [dataset_processors.preprocess_text(essay) for essay in essays]

    # Encode them using the tokenizer
    encoded_essay = tokenizer(essays, truncation=True)
    
    # Add the labels
    labels_batch = {key: row[key] for key in row.keys() if key in labels}

    # Create numpy array of batch and labels
    labels_matrix = np.zeros((len(essays), len(labels)))

    # Fill the array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    # Return the encoding
    encoded_essay["labels"] = labels_matrix.tolist()
    
    return encoded_essay

# Perform the preprocessing
full_dataset_dict = full_dataset_dict.map(
    preprocess_text, batched = True, 
    remove_columns = full_dataset_dict['train'].column_names
)

Map:   0%|          | 0/1480 [00:00<?, ? examples/s]

Map: 100%|██████████| 493/493 [00:01<00:00, 332.83 examples/s]


In [10]:
#print(full_dataset_dict['train'][0].keys())
print(full_dataset_dict['train'][0])
#print(full_dataset_dict['train'][0]['labels'])
#tokenizer.decode(full_dataset_dict['train'][5]['input_ids'])
#[id2label[idx] for idx, label in enumerate(full_dataset_dict['train'][5]['labels']) if label == 1.0]

{'input_ids': [101, 1045, 2572, 5458, 2085, 1012, 1045, 2123, 1005, 1056, 2113, 2054, 1045, 2323, 2831, 2055, 1012, 1045, 2066, 2023, 8775, 1012, 4687, 2043, 2009, 1005, 1055, 2349, 1029, 19031, 3775, 24471, 3240, 2003, 1996, 2087, 3376, 2711, 1045, 2113, 1012, 1045, 2293, 2017, 1012, 1045, 2812, 1045, 2293, 2014, 2007, 2035, 1997, 2026, 2540, 1010, 2568, 1010, 2303, 1010, 1998, 3969, 1012, 1045, 2066, 6825, 1999, 2152, 2082, 1012, 2009, 2001, 2200, 5875, 1998, 3167, 1012, 2008, 9891, 2111, 1999, 2030, 2061, 1045, 2228, 1012, 6825, 2003, 1996, 2833, 1997, 1996, 2712, 1012, 2026, 18328, 1005, 1055, 1037, 6881, 2080, 1012, 2002, 11651, 1037, 6045, 2000, 3422, 16608, 1051, 1005, 9848, 1012, 2026, 2060, 18328, 2288, 2010, 3274, 2013, 12418, 2651, 1012, 1045, 2066, 19031, 3775, 1012, 1045, 2089, 2025, 2022, 1037, 6047, 2158, 1010, 2021, 1045, 2113, 2054, 2293, 2003, 1012, 2619, 1005, 1055, 2183, 2000, 2022, 3374, 1010, 2066, 2035, 2017, 12566, 1998, 2035, 2017, 11754, 2063, 1998, 2035, 2017

In [11]:
full_dataset_dict.set_format("torch", device="cuda:0")
model = model.to("cuda:0")

In [12]:
from transformers import TrainingArguments, Trainer

batch_size = 16
learning_rate = 2e-5
epochs = 10
metric_name = "accuracy"

print(learning_rate)

2e-05


In [13]:
args = TrainingArguments(
    saved_model,
    evaluation_strategy = "epoch",
    save_strategy = "no",
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs = epochs,
    weight_decay = 0.01,
    metric_for_best_model = metric_name
)

c:\Users\User\anaconda3\envs\ml_projects\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
print(labels)

['EXT', 'NEU', 'AGR', 'CON', 'OPN']


In [15]:
from sklearn.metrics import accuracy_score
import torch

def multi_label_metrics(pred_logits, gold_labels):

    # Our threshold
    threshold = 0.5

    # Apply sigmoid to logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(pred_logits))

    # Convert predictions to integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # Compute metrics
    y_true = gold_labels

    metrics = {
        f"{id2label[i]} - accuracy": accuracy_score(y_true[:, i], y_pred[:, i]) 
        for i in range(len(labels))
    }

    overall_accuracy = accuracy_score(y_true, y_pred)

    # Store and return as dictionary
    metrics['accuracy'] = overall_accuracy
    
    return metrics

def compute_metrics(p):

    # Get the type of predictions
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

    # Compute the results
    results = multi_label_metrics(preds, p.label_ids)

    return results

In [16]:
# forward pass (testing)
outputs = model(
    input_ids=full_dataset_dict['train']['input_ids'][0].unsqueeze(0), 
    attention_mask=full_dataset_dict['train']['attention_mask'][0].unsqueeze(0),
    labels=full_dataset_dict['train'][0]['labels'].unsqueeze(0)
)

outputs

SequenceClassifierOutput(loss=tensor(0.7369, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0444, -0.0161, -0.0475,  0.1298, -0.2149]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
# Initialize the trainer before training
trainer = Trainer(
    model,
    args,
    train_dataset=full_dataset_dict["train"],
    eval_dataset=full_dataset_dict["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

  1%|          | 501/45970 [00:47<1:03:49, 11.87it/s]

{'loss': 0.691, 'grad_norm': 0.5171191096305847, 'learning_rate': 1.9782466826190996e-05, 'epoch': 0.11}


  2%|▏         | 1002/45970 [01:37<1:04:12, 11.67it/s]

{'loss': 0.6879, 'grad_norm': 0.6280295848846436, 'learning_rate': 1.956493365238199e-05, 'epoch': 0.22}


  3%|▎         | 1502/45970 [02:26<1:00:38, 12.22it/s]

...

 99%|█████████▉| 45503/45970 [1:14:47<00:35, 13.32it/s]

{'loss': 0.3579, 'grad_norm': 5.9566731452941895, 'learning_rate': 2.0448118338046557e-07, 'epoch': 9.9}


                                                       
100%|██████████| 45970/45970 [1:16:09<00:00, 10.06it/s]

{'eval_loss': 1.163111686706543, 'eval_EXT - accuracy': 0.5138710055015803, 'eval_NEU - accuracy': 0.5157438838815405, 'eval_AGR - accuracy': 0.5143782433961528, 'eval_CON - accuracy': 0.5131296578095127, 'eval_OPN - accuracy': 0.5310000390182996, 'eval_accuracy': 0.04210074524952202, 'eval_runtime': 37.6444, 'eval_samples_per_second': 680.818, 'eval_steps_per_second': 42.556, 'epoch': 10.0}
{'train_runtime': 4569.2885, 'train_samples_per_second': 160.938, 'train_steps_per_second': 10.061, 'train_loss': 0.5155778935921822, 'epoch': 10.0}


TrainOutput(global_step=45970, training_loss=0.5155778935921822, metrics={'train_runtime': 4569.2885, 'train_samples_per_second': 160.938, 'train_steps_per_second': 10.061, 'total_flos': 9631412851951230.0, 'train_loss': 0.5155778935921822, 'epoch': 10.0})

In [20]:
trainer.save_model(saved_model)